# Neuromodulation example

This neuromodulation creates a small network of neurons. We also use the reaction diffusion model by Anu G Nair 2015.

To generate the ```reaction_diffusion.json``` file in ```data/dspn_rxd``` from the xml file we run ```data/convert_sbml_to_json.sh```

To get the RxD functionality of the ```DA_syn``` we must specify the "RxD" block under channel_parameters in the connectivity block of the network configuration. See ```data/connectivity.json```

```
    "channel_parameters":
		"RxD": {
		    "species_name": "DA",
		    "flux_variable": "open",
		    "region": "internal"
		},

     ...
     }
```

In [1]:
import os
from snudda import Snudda

neuron_path = os.path.join("data", "dspn_rxd")
network_path = os.path.join("networks", "neuromodulation_example_anu")
connectivity_path = os.path.join("data", "connectivity.json")

snudda = Snudda(network_path=network_path)
snudda.init_tiny(neuron_paths=neuron_path, neuron_names="neuron_1", number_of_neurons=[2], 
                 connection_config=connectivity_path, random_seed=12345)
snudda.create_network()

Reading connectivity from data/connectivity.json
Adding neurons: neuron_1 from dir data/dspn_rxd
Writing networks/neuromodulation_example_anu/network-config.json
Placing neurons
Network path: networks/neuromodulation_example_anu
Reading SNUDDA_DATA=None from networks/neuromodulation_example_anu/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from networks/neuromodulation_example_anu/network-synapses.hdf5
No n_putative_points and putative_density, setting n_putative_points = 63
(this must be larger than the number of neurons you want to place)
Generating 63 points for networks/neuromodulation_example_anu/mesh/Cube-cube-mesh-2.917951293943981e-05.obj
Filtering, keeping inside points: 4 / 26
neuron_name = 'neuron_1', num = 2, neuron_path = 'data/dspn_rxd'
stop_parallel disabled, to keep pool running.

Execution time: 0.0s
Touch detection
Network path: networks/neuromodulation_example_anu
Reading SNUDDA_DATA=None from networks/neuromodulation_example_an

In [2]:
input_config = os.path.join("data", "input.json")
snudda.setup_input(input_config=input_config)

Setting up inputs, assuming input.json exists
Reading SNUDDA_DATA=None from networks/neuromodulation_example_anu/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from networks/neuromodulation_example_anu/network-synapses.hdf5
Writing input spikes to networks/neuromodulation_example_anu/input-spikes.hdf5
Reading SNUDDA_DATA=None from networks/neuromodulation_example_anu/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from networks/neuromodulation_example_anu/network-synapses.hdf5
Writing spikes to networks/neuromodulation_example_anu/input-spikes.hdf5
stop_parallel disabled, to keep pool running.

Execution time: 1.1s


In [3]:
mech_dir = os.path.join("data", "mechanisms")
sim = snudda.simulate(time=0, mech_dir=mech_dir, verbose=True)

Using input file networks/neuromodulation_example_anu/input-spikes.hdf5
NEURON mechanisms already compiled, make sure you have the correct version of NEURON modules.
If you delete x86_64, aarch64, arm64 directories (or nrnmech.dll) then you will force a recompilation of the modules.
Reading SNUDDA_DATA=None from networks/neuromodulation_example_anu/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from networks/neuromodulation_example_anu/network-synapses.hdf5
Using network_file: networks/neuromodulation_example_anu/network-synapses.hdf5
Using input_file: networks/neuromodulation_example_anu/input-spikes.hdf5
Using output_file: networks/neuromodulation_example_anu/simulation/output.hdf5
Using logFile: networks/neuromodulation_example_anu/log/network-simulation-log.txt-0
Worker 0 : Loading network from networks/neuromodulation_example_anu/network-synapses.hdf5
Loading config file networks/neuromodulation_example_anu/network-config.json
0 : Memory statu

In [4]:
sim.add_rxd_concentration_recording(species="PKA", neuron_id=0,
                                    region="soma_internal",
                                    sec_type="soma",
                                    sec_id=0,
                                    sec_x=0.5)

In [ ]:
sim.run(t=500)

Running simulation for 0.5 s
Running Neuron simulator 500 ms, with dt=0.025
1% done. Elapsed: 2.0 s, estimated time left: 194.7 s
2% done. Elapsed: 3.1 s, estimated time left: 150.6 s
3% done. Elapsed: 4.2 s, estimated time left: 135.6 s
4% done. Elapsed: 5.3 s, estimated time left: 127.4 s
5% done. Elapsed: 6.4 s, estimated time left: 121.9 s
6% done. Elapsed: 7.5 s, estimated time left: 117.9 s
7% done. Elapsed: 8.6 s, estimated time left: 114.8 s
8% done. Elapsed: 9.8 s, estimated time left: 112.3 s
9% done. Elapsed: 10.9 s, estimated time left: 110.1 s
10% done. Elapsed: 12.0 s, estimated time left: 108.0 s
11% done. Elapsed: 13.1 s, estimated time left: 106.2 s
12% done. Elapsed: 14.2 s, estimated time left: 104.4 s
13% done. Elapsed: 15.4 s, estimated time left: 102.8 s
14% done. Elapsed: 16.5 s, estimated time left: 101.2 s
15% done. Elapsed: 17.6 s, estimated time left: 99.7 s
16% done. Elapsed: 18.7 s, estimated time left: 98.2 s
17% done. Elapsed: 19.8 s, estimated time left:

In [ ]:
sim.record.write()

## Load the data and plot

In [ ]:
from snudda.utils import SnuddaLoadNetworkSimulation

output_file = os.path.join(network_path, "simulation", "output.hdf5")
nd = SnuddaLoadNetworkSimulation(output_file)
time = nd.get_time()
data_pka = nd.get_data("PKA", 0)[0][0]

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time, data_pka, label="PKA")
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
plt.legend()
plt.show()

In [ ]:
from snudda.plotting import SnuddaPlotSpikeRaster2
fig_file_raster = f"spike-raster.png"

time_range_zoom = (0,0.5)
spr = SnuddaPlotSpikeRaster2(network_path=network_path)

spr.plot_spike_raster(fig_file=fig_file_raster, time_range=time_range_zoom)

In [ ]:
%matplotlib inline

sim_file = os.path.join(network_path, "simulation", "output.hdf5")

from snudda.plotting.plot_traces import PlotTraces
pt = PlotTraces(output_file=sim_file)
# Use trace_id to specify which traces
ax = pt.plot_traces(offset=0, time_range=(0,0.5),fig_size=(10,4))

## TODO!! Also add ability to have external synapses affect RxD concentrations